# AWS Test S3- Glue- RDSinstance- IAM-securityGrups- Postgres-Lambda- API RESTful
##### flujo de trabajo:
- Almacenamiento de Datos: Uso **pgvector en Postgres** para almacenar datos médicos (con embeddings vectoriales).
- ETL con AWS Glue: AWS glue para leer datos de s3, transformarlos y cargarlos en postgress.
- Consultas de similaridad: uso consultas SQL con **pgvector** para encontrar enfermedades similares basadas en sus embeddings.
- Lambda para API: Implementar una API RESTful en AWS Lambda para hacer consultas en tiempo real.

In [29]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
import boto3
from awsglue.utils import getResolvedOptions
from awsglue.job import Job
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

You are already connected to a glueetl session 1bc1bf08-ef3a-4246-b6c8-6f12348af977.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 1bc1bf08-ef3a-4246-b6c8-6f12348af977.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 1bc1bf08-ef3a-4246-b6c8-6f12348af977.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 1bc1bf08-ef3a-4246-b6c8-6f12348af977.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [30]:
# Obtener los argumentos necesarios
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

GlueArgumentError: the following arguments are required: --JOB_NAME


In [31]:
# Crear o reutilizar el SparkContext y GlueContext
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

In [32]:
# Crear el objeto 'job' e inicializarlo
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

NameError: name 'args' is not defined


 ### Ingesta y transformación de datos

In [33]:
# Leer el CSV desde S3
df = spark.read.option("header", "true").csv("s3://source-embeddings-ohio/medical_data.csv")

# Transformar la columna de embeddings (convertir el texto a un array de float)
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

def parse_embedding(embedding_str):
    return list(map(float, embedding_str.strip("[]").split(", ")))

parse_embedding_udf = udf(parse_embedding, ArrayType(FloatType()))
df = df.withColumn("embedding", parse_embedding_udf(df["embedding"]))


### Conexion en tabla en PostgreSQL

In [34]:
# Conexión a la base de datos Aurora PostgreSQL
connection_options = {
    "url": "jdbc:postgresql://db-identifier-aws-postgre.cj6s66gsqow2.us-east-2.rds.amazonaws.com:5432/sampleDB",
    "user": "postgres",
    "password": "Adri2319$",
    "dbtable": "test_embeddings", 
    "driver": "org.postgresql.Driver"
}

### carga

In [36]:
# Cargar datos en la base de datos Aurora
df.write.format("jdbc").options(**connection_options).mode("append").save()

job.commit()

**DONE!**